In [ ]:
import pandas as pd
import rasterio
import torch
import torch.nn as nn
import torch.optim as optim

# Load the FOWOT input data into a Pandas dataframe
data = pd.read_csv('fowot_input.csv')

# Get a list of unique cultivars in the data
unique_cultivars = data['cultivar'].unique()

# Train a separate FOWOT model for each unique cultivar
models = {}
for cultivar in unique_cultivars:
    # Create a mask to select only the rows with the current cultivar
    mask = data['cultivar'] == cultivar

    # Split the input data and targets into train and validation sets
    train_data = data[mask].sample(frac=0.8, random_state=1)
    val_data = data[mask].drop(train_data.index)

    # Read geotiffs as input data
    with rasterio.open('input_geotiff.tif') as src:
        train_inputs = []
        for idx, row in train_data.iterrows():
            x, y = src.index(row['longitude'], row['latitude'])
            value = src.read(1, window=((y, y+1), (x, x+1)))[0][0]
            train_inputs.append([value, ...]) # Other input features

        train_inputs = torch.tensor(train_inputs, dtype=torch.float32)

        val_inputs = []
        for idx, row in val_data.iterrows():
            x, y = src.index(row['longitude'], row['latitude'])
            value = src.read(1, window=((y, y+1), (x, x+1)))[0][0]
            val_inputs.append([value, ...]) # Other input features

        val_inputs = torch.tensor(val_inputs, dtype=torch.float32)

    train_targets = torch.tensor(train_data['vulnerability'].values, dtype=torch.float32)
    val_targets = torch.tensor(val_data['vulnerability'].values, dtype=torch.float32)

    # Define the FOWOT model with geotiff input
    class FOWOTModel(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(FOWOTModel, self).__init__()
            self.linear1 = nn.Linear(input_size, hidden_size)
            self.linear2 = nn.Linear(hidden_size, output_size)
            self.relu = nn.ReLU()

        def forward(self, x):
            out = self.linear1(x)
            out = self.relu(out)
            out = self.linear2(out)
            return out

    model = FOWOTModel(train_inputs.size(1), 64, 1)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train the FOWOT model
    for epoch in range(1000):
        outputs = model(train_inputs)
        loss = criterion(outputs, train_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print('Cultivar: {}, Epoch [{}/{}], Loss: {:.4f}'.format(cultivar, epoch+1, 1000, loss.item()))

    # Evaluate the FOWOT model on the validation set
    with torch.no_grad():
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_targets)

    # Save the FOWOT model for the current cultivar
    models[cultivar] = model

print('Training complete. Models saved.')
